In [ ]:
from zipfile import ZipFile

with ZipFile("/content/drive/MyDrive/face recognition dataset.zip","r") as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np

from torchvision import utils
from torchvision import datasets, models, transforms
import os

In [ ]:
data_transforms = {
    "test" : transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    "train" : transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

In [ ]:
data_dir = "/content/face recognition dataset"

In [ ]:
image_dataset = {x: datasets.ImageFolder(os.path.join(data_dir,x),
                                        data_transforms[x])
                for x in ["test", "train"]}

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_dataset[x], batch_size=16,
                                              shuffle = True,num_workers = 2)
                           for x in ["test", "train"]}


In [ ]:
len(dataloaders["train"])

14

In [ ]:
dataset_sizes = {x: len(image_dataset[x]) for x in ["test","train"]}
class_names = image_dataset["train"].classes

In [ ]:
class_names

['not_vedansh', 'vedansh']

In [ ]:
pip install facenet-pytorch

     |████████████████████████████████| 1.9MB 8.7MB/s 


In [ ]:
import torchvision
model=torchvision.models.mobilenet_v2(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
for param in model.parameters():
    param.requires_grad=True

classifier = nn.Sequential(nn.Linear(1000,256),nn.ReLU(),nn.Dropout(0.25),nn.Linear(256,2),nn.Sigmoid())
model.fc = classifier

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_losses = []
validation_losses = []

for epoch in range(1,11):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for input,target in dataloaders["train"]:
        input = input.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    

    model.eval()
    for input,target in dataloaders["test"]:
        input = input.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output,target)

        valid_loss += loss.item()
    
    exp_lr_scheduler.step()
    
    # calculate average losses
    train_loss = train_loss/len(dataloaders["train"])
    valid_loss = valid_loss/len(dataloaders["test"])
    train_losses.append(train_loss)
    validation_losses.append(valid_loss)

    print(f"Epoch {epoch}: \t Training Loss:  {train_loss} \t Validation Loss:  {valid_loss}")

Epoch 1: 	 Training Loss:  1.8115139445601796 	 Validation Loss:  0.6056193310942035
Epoch 2: 	 Training Loss:  0.08939583352808508 	 Validation Loss:  0.17456076250770516
Epoch 3: 	 Training Loss:  0.18309665166764585 	 Validation Loss:  0.007493235636502504
Epoch 4: 	 Training Loss:  0.01119290579767299 	 Validation Loss:  0.0004959684447385371
Epoch 5: 	 Training Loss:  0.21438479067104968 	 Validation Loss:  0.007655012906373789
Epoch 6: 	 Training Loss:  0.04453608107827937 	 Validation Loss:  0.001401048890935878
Epoch 7: 	 Training Loss:  0.02042256776270993 	 Validation Loss:  0.00037008270737715065
Epoch 8: 	 Training Loss:  0.1568110439192034 	 Validation Loss:  0.0006696553221748521
Epoch 9: 	 Training Loss:  0.0005037503433413804 	 Validation Loss:  0.0009875541970056172
Epoch 10: 	 Training Loss:  0.0008285718625172324 	 Validation Loss:  0.0006029401896133398


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data[0].to(device),data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the validation set is: {100*correct/total}%")

Accuracy of the network on the validation set is: 100.0%


In [ ]:
torch.save(model.state_dict(), "/content/model_Inc.pth")